In [2]:
import tensorflow as tf
import keras

Using TensorFlow backend.


In [3]:
tf.__version__

'1.10.0'

# Keras for Neural Networks - Guided Example

Here we're going to work through a classic Machine Learning problem - digit recognition. This data is referred to as the MNIST dataset, which stands for Modified National Institute of Standards and Technology, and represents probably the most used dataset in the world for advanced machine learning techniques (though the iris dataset would be a close second). Here we're forgoing a more business focussed dataset for a few reasons. Firstly, this dataset is the most written about dataset in these topics - you'll easily find other guides using pure TensorFlow or other tools like Theano to solve the same problem with the same class of models. Similarly, you can also easily find several different kinds of neural networks being used to solve this problem. This will be valuable as you try to expand your knowledge of different kinds of layers and combinations.

We'll be building our code off of the examples provided in the Keras documentation, and found in full on its [creator's github](https://github.com/fchollet/keras/tree/master/examples). 

Our goal here will be simple but multifaceted. Overall we are going to use the MNIST dataset and neural networks to classify handwritten numbers as the proper digits. This will be thought of as a multi-class classification problem, specifically with 10 classes (one for each possible digit).

However, we will use this to teach a few new kinds of neural network compositions, creating three different styles of network and discussing their relative advantages and disadvantages. Through this we will delve a little deeper into neural network theory.

But before we go too far, let's actually look at the data.

## MNIST DATA

In [172]:
# Import the dataset
from keras.datasets import mnist

# Import various componenets for model building
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop, Adam, Adagrad, Adadelta, Nadam

# Import the backend
from keras import backend as K

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
x_train

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

When you look at this data you'll notice its organization structure is not images. We don't actually see any pictures of digits here. Instead, what we have is values of pixels, a simple way of converting images into numeric data on which we can train a model.

However, this still doesn't look like most of the data we've worked with previously. It's not a single table, but rather a different, higher dimensionality structure. It is often described as a set of clouds, each cloud representing an image. The cloud contains columns of values, representing the darkness of pixels. That's great, but not an easy or meaningful dataset on which to directly train a model. The darkness of the second pixel in the third column isn't likely linearly related to likelihood the cloud represents a certain digit. Instead, we need to find meaningful patterns within our clouds, creating models off of those patterns.

This is exactly what neural networks are good at. Multiple layers will allow us to transform this clouds full of values into meaningful vectors containing the information we need to be able to create a model, admittedly in an unlabeled and unsupervised fashion. Our output, however, will be labels for each of the clouds, giving us predictions as to what digit they are meant to represent.

Let's get started.

## Multi Layer Perceptron

Let's start with a kind of neural network we've seen before: a multi-layer perceptron. Recall from our previous neural networks sections that this is a set of perceptron models organized into layers, one layer feeding into the next.

To do this, we will first need to reshape our data into flat vectors for each digit. We'll also need to convert our outcome to a matrix of binary variables, rather than the digit.

In [6]:
# Change shape 
# Note that our images are 28*28 pixels, so in reshaping to arrays we want
# 60,000 arrays of length 784, one for each image
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

# Convert to float32 for type consistency
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize values to 1 from 0 to 255 (256 values of pixels)
x_train /= 255
x_test /= 255

# Print sample sizes
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
# So instead of one column with 10 values, create 10 binary columns
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

60000 train samples
10000 test samples


In [8]:
y_test[0]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32)

Great. Now we can create our model. We'll do this using dense layers and dropouts. Dense layers are simply fully connected layers with a given number of perceptrons. Dropout drops a certain portion of our perceptrons in order to prevent overfitting. Our activation function, `relu` stands for Rectified Linear Unit, which is standard but can be read about more [here](https://en.wikipedia.org/wiki/Rectifier_(neural_networks).

In [9]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(64, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


Now we have a model. This we can use to accomplish our wildest dreams of data modeling, or at least predict some digits from pixel data. To do that we will use epochs, effectively iterations of the model, improving based on what it learned previously. Batch size is the number of samples to use in each step improving the model and will affect speed, but also slightly negatively impact accuracy (learning in bigger steps will affect what your model learns).

Note that we are going with 64 perceptron wide layers, this is relatively arbitrary, though units within the $2^x$ series will parallelize more efficiently. Also note that our number of parameters is the product of our input width plus one and our layer width. This reflects the number of weights we're creating in that layer.

In [11]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 1s 21us/step - loss: 0.4448 - acc: 0.8678 - val_loss: 0.1994 - val_acc: 0.9390
Epoch 2/10
60000/60000 [==============================] - 1s 15us/step - loss: 0.2115 - acc: 0.9369 - val_loss: 0.1380 - val_acc: 0.9581
Epoch 3/10
60000/60000 [==============================] - 1s 14us/step - loss: 0.1615 - acc: 0.9520 - val_loss: 0.1214 - val_acc: 0.9632
Epoch 4/10
60000/60000 [==============================] - 1s 15us/step - loss: 0.1358 - acc: 0.9587 - val_loss: 0.1105 - val_acc: 0.9687
Epoch 5/10
60000/60000 [==============================] - 1s 15us/step - loss: 0.1173 - acc: 0.9640 - val_loss: 0.0981 - val_acc: 0.9719
Epoch 6/10
60000/60000 [==============================] - 1s 15us/step - loss: 0.1093 - acc: 0.9669 - val_loss: 0.0973 - val_acc: 0.9735
Epoch 7/10
60000/60000 [==============================] - 1s 15us/step - loss: 0.0993 - acc: 0.9700 - val_loss: 0.0912 - val_acc

That did impressively well for such a simple neural network, with each epoch training in about 1 second on this machine and giving us an accuracy in the high 90's. But what else can we do? Let's let our model get much more complicated by introducting convolution.

## Convolutional Neural Networks

Before we go any further, do you recall that we've talked about how complex neural networks can get, and the degree of computational complexity that entails? Well, here we're going to finally truly experience that complexity, so be careful about rerunning this code. It will take some serious time (potentially on the order of hours) to run.

Now that that's out of the way, let's talk convolution. First, a simple definition. Convolution basically takes your data and creates overlapping subsegments testing for a given feature in a set of spaces and upon which it develops its model.

Let's extend that definition since it's incredibly dense.

First, you have to define a shape of your input data. This can theoretically be in any number of dimensions, though for our image example we will use 2d, since images are in two dimensions. This is also why you'll see 2D in some of our layer definitions (though more on that later). Our first chunk of code after loading the data does this reshaping (with a conditional on the data format).

Over that shaped data, we then create our tiles, also called __kernels__. These kernels are like little windows, that will look over subsets of the data of a given size. In the example below we create 3x3 kernels, which run overlapping over the whole 28x28 input looking for features. That is the convolutional layer, a way of searching for a subpattern over the whole of the image. We can chain multiple of these convolutional layers together, with the below example having two.

Next comes a pooling layer. This is a _downsampling_ technique, which effectively serves to reduce sample size and simplify later processes. For each value generated by our convolutional layers, it looks over the grid in _non_-overlapping segments and takes the maximum value of those outputs. It's not the feautres exact location then that matters, but its approximate or relative location. After pooling you will want to flatten the data back out, so that it can be put into dense layers as we did in MLP.

In [12]:
# input image dimensions, from our data
img_rows, img_cols = 28, 28
num_classes = 10

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Building the Model
model = Sequential()
# First convolutional layer, note the specification of shape
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 87s 1ms/step - loss: 0.2621 - acc: 0.9202 - val_loss: 0.0530 - val_acc: 0.9825
Epoch 2/10
60000/60000 [==============================] - 87s 1ms/step - loss: 0.0861 - acc: 0.9744 - val_loss: 0.0370 - val_acc: 0.9862
Epoch 3/10
60000/60000 [==============================] - 86s 1ms/step - loss: 0.0652 - acc: 0.9807 - val_loss: 0.0347 - val_acc: 0.9877
Epoch 4/10
60000/60000 [==============================] - 86s 1ms/step - loss: 0.0545 - acc: 0.9841 - val_loss: 0.0360 - val_acc: 0.9883
Epoch 5/10
60000/60000 [==============================] - 95s 2ms/step - loss: 0.0482 - acc: 0.9856 - val_loss: 0.0325 - val_acc: 0.9895
Epoch 6/10
60000/60000 [==============================] - 98s 2ms/step - loss: 0.0428 - acc: 0.9872 - val_loss: 0.0279 - val_acc: 0.9908
Epoch 7/10
60000/60000 [==============================]

Now that is incredibly impressive accuracy. 99% is really exceptional, but it did take a long time to get there. Such are the costs of convolution.

There is one more classic construction of a neural network: Recurrent, which we'll give quick mention.

## Hierarchical Recurrent Neural Networks

So far when we've talked about neural networks we've talked about them as feedforward: data flows in one direction until it reaches the end. Recurrent neural networks do not obey that directional logic, instead letting the data cycle through the network.

However, to do this we have to abandon the sequential model building we've done so far and things can get much more complicated. You have to use recurrent layers and often time distribution (which handles the extra dimension created through the LTSM layer, as a time dimension) to get these things running. You can find an example of a hierarchical recurrent network below (via the link [here](https://github.com/fchollet/keras/blob/master/examples/mnist_hierarchical_rnn.py)). When you get comfortable with networks as they exist in Keras for both convolution and MLP, start exploring recurrence. Note that this will take an even longer time than the previous ones should you choose to run it again.

In [13]:
# Training parameters.
batch_size = 64
num_classes = 10
epochs = 3

# Embedding dimensions.
row_hidden = 32
col_hidden = 32

# The data, shuffled and split between train and test sets.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshapes data to 4D for Hierarchical RNN.
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Converts class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

row, col, pixel = x_train.shape[1:]

# 4D input.
x = Input(shape=(row, col, pixel))

# Encodes a row of pixels using TimeDistributed Wrapper.
encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# Encodes columns of encoded rows.
encoded_columns = LSTM(col_hidden)(encoded_rows)

# Final predictions and model.
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Training.
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

# Evaluation.
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 93s 2ms/step - loss: 1.0798 - acc: 0.6243 - val_loss: 0.5436 - val_acc: 0.8076
Epoch 2/3
60000/60000 [==============================] - 94s 2ms/step - loss: 0.4460 - acc: 0.8496 - val_loss: 0.3011 - val_acc: 0.9050
Epoch 3/3
60000/60000 [==============================] - 93s 2ms/step - loss: 0.2565 - acc: 0.9204 - val_loss: 0.2170 - val_acc: 0.9311
Test loss: 0.21700494042634963
Test accuracy: 0.9311


You should now be comfortable building some neural networks, but let's see if you can improve them!

# Drill: 99% MLP

We have the MLP above, which runs reasonably quickly. Copy that code down here and see if you can tune it to achieve 99% accuracy with a Multi-Layer Perceptron. Does it run faster than the recurrent or concolutional neural nets?

In [173]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Change shape 
# Note that our images are 28*28 pixels, so in reshaping to arrays we want
# 60,000 arrays of length 784, one for each image
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

# Convert to float32 for type consistency
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize values to 1 from 0 to 255 (256 values of pixels)
x_train /= 255
x_test /= 255

# Print sample sizes
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
# So instead of one column with 10 values, create 10 binary columns
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

60000 train samples
10000 test samples


## Hidden Layers: 256, dropout: 0.2

In [199]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(256, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_250 (Dense)            (None, 256)               200960    
_________________________________________________________________
dropout_170 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_251 (Dense)            (None, 256)               65792     
_________________________________________________________________
dropout_171 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_252 (Dense)            (None, 10)                2570      
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________


In [200]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 6s 108us/step - loss: 0.2975 - acc: 0.9105 - val_loss: 0.1290 - val_acc: 0.9622
Epoch 2/10
60000/60000 [==============================] - 3s 54us/step - loss: 0.1280 - acc: 0.9612 - val_loss: 0.0967 - val_acc: 0.9705
Epoch 3/10
60000/60000 [==============================] - 3s 57us/step - loss: 0.0947 - acc: 0.9709 - val_loss: 0.0819 - val_acc: 0.9758
Epoch 4/10
60000/60000 [==============================] - 3s 56us/step - loss: 0.0774 - acc: 0.9764 - val_loss: 0.0810 - val_acc: 0.9759
Epoch 5/10
60000/60000 [==============================] - 3s 55us/step - loss: 0.0659 - acc: 0.9801 - val_loss: 0.0822 - val_acc: 0.9776
Epoch 6/10
60000/60000 [==============================] - 3s 57us/step - loss: 0.0587 - acc: 0.9821 - val_loss: 0.0737 - val_acc: 0.9811
Epoch 7/10
60000/60000 [==============================] - 3s 56us/step - loss: 0.0547 - acc: 0.9839 - val_loss: 0.0761 - val_ac

## Hidden Layers: 256, dropout: 0.2, extra hidden layer

In [176]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(256, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_215 (Dense)            (None, 256)               200960    
_________________________________________________________________
dropout_146 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_216 (Dense)            (None, 256)               65792     
_________________________________________________________________
dropout_147 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_217 (Dense)            (None, 256)               65792     
_________________________________________________________________
dropout_148 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_218 (Dense)            (None, 10)                2570      
Total para

In [177]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s 118us/step - loss: 0.3122 - acc: 0.9041 - val_loss: 0.1153 - val_acc: 0.9637
Epoch 2/10
60000/60000 [==============================] - 4s 68us/step - loss: 0.1361 - acc: 0.9603 - val_loss: 0.1286 - val_acc: 0.9600
Epoch 3/10
60000/60000 [==============================] - 4s 67us/step - loss: 0.1012 - acc: 0.9706 - val_loss: 0.0820 - val_acc: 0.9767
Epoch 4/10
60000/60000 [==============================] - 4s 66us/step - loss: 0.0869 - acc: 0.9749 - val_loss: 0.0773 - val_acc: 0.9769
Epoch 5/10
60000/60000 [==============================] - 4s 67us/step - loss: 0.0759 - acc: 0.9782 - val_loss: 0.0758 - val_acc: 0.9793
Epoch 6/10
60000/60000 [==============================] - 4s 67us/step - loss: 0.0682 - acc: 0.9804 - val_loss: 0.0898 - val_acc: 0.9778
Epoch 7/10
60000/60000 [==============================] - 4s 67us/step - loss: 0.0620 - acc: 0.9826 - val_loss: 0.0739 - val_ac

## Hidden Layers: 256, dropout: 0.2, two extra hidden layers

In [178]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(256, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_219 (Dense)            (None, 256)               200960    
_________________________________________________________________
dropout_149 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_220 (Dense)            (None, 256)               65792     
_________________________________________________________________
dropout_150 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_221 (Dense)            (None, 256)               65792     
_________________________________________________________________
dropout_151 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_222 (Dense)            (None, 10)                2570      
Total para

In [179]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s 115us/step - loss: 0.3125 - acc: 0.9051 - val_loss: 0.1347 - val_acc: 0.9570
Epoch 2/10
60000/60000 [==============================] - 4s 69us/step - loss: 0.1304 - acc: 0.9598 - val_loss: 0.0919 - val_acc: 0.9715
Epoch 3/10
60000/60000 [==============================] - 4s 68us/step - loss: 0.0989 - acc: 0.9703 - val_loss: 0.0759 - val_acc: 0.9772
Epoch 4/10
60000/60000 [==============================] - 4s 68us/step - loss: 0.0826 - acc: 0.9754 - val_loss: 0.0786 - val_acc: 0.9776
Epoch 5/10
60000/60000 [==============================] - 4s 68us/step - loss: 0.0729 - acc: 0.9775 - val_loss: 0.0778 - val_acc: 0.9785
Epoch 6/10
60000/60000 [==============================] - 4s 67us/step - loss: 0.0669 - acc: 0.9809 - val_loss: 0.0814 - val_acc: 0.9786
Epoch 7/10
60000/60000 [==============================] - 4s 68us/step - loss: 0.0605 - acc: 0.9824 - val_loss: 0.0814 - val_ac

## Hidden Layers: 784, dropout: 0.2

In [180]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(784, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.2))
model.add(Dense(784, activation='relu'))
model.add(Dropout(0.2))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_223 (Dense)            (None, 784)               615440    
_________________________________________________________________
dropout_152 (Dropout)        (None, 784)               0         
_________________________________________________________________
dense_224 (Dense)            (None, 784)               615440    
_________________________________________________________________
dropout_153 (Dropout)        (None, 784)               0         
_________________________________________________________________
dense_225 (Dense)            (None, 10)                7850      
Total params: 1,238,730
Trainable params: 1,238,730
Non-trainable params: 0
_________________________________________________________________


In [181]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 11s 184us/step - loss: 0.2336 - acc: 0.9283 - val_loss: 0.1084 - val_acc: 0.9689
Epoch 2/10
60000/60000 [==============================] - 8s 132us/step - loss: 0.0952 - acc: 0.9715 - val_loss: 0.0891 - val_acc: 0.9736
Epoch 3/10
60000/60000 [==============================] - 8s 131us/step - loss: 0.0712 - acc: 0.9790 - val_loss: 0.0770 - val_acc: 0.9783
Epoch 4/10
60000/60000 [==============================] - 8s 132us/step - loss: 0.0577 - acc: 0.9832 - val_loss: 0.0767 - val_acc: 0.9788
Epoch 5/10
60000/60000 [==============================] - 8s 133us/step - loss: 0.0495 - acc: 0.9859 - val_loss: 0.0659 - val_acc: 0.9831
Epoch 6/10
60000/60000 [==============================] - 8s 133us/step - loss: 0.0421 - acc: 0.9876 - val_loss: 0.0901 - val_acc: 0.9799
Epoch 7/10
60000/60000 [==============================] - 8s 132us/step - loss: 0.0384 - acc: 0.9892 - val_loss: 0.0881 -

## Hidden Layers: 256, dropout: 0.5

In [182]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(256, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_226 (Dense)            (None, 256)               200960    
_________________________________________________________________
dropout_154 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_227 (Dense)            (None, 256)               65792     
_________________________________________________________________
dropout_155 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_228 (Dense)            (None, 10)                2570      
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________


In [183]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 6s 98us/step - loss: 0.4277 - acc: 0.8683 - val_loss: 0.1537 - val_acc: 0.9538
Epoch 2/10
60000/60000 [==============================] - 3s 53us/step - loss: 0.2164 - acc: 0.9369 - val_loss: 0.1207 - val_acc: 0.9651
Epoch 3/10
60000/60000 [==============================] - 3s 53us/step - loss: 0.1749 - acc: 0.9502 - val_loss: 0.1047 - val_acc: 0.9698
Epoch 4/10
60000/60000 [==============================] - 3s 53us/step - loss: 0.1549 - acc: 0.9563 - val_loss: 0.0961 - val_acc: 0.9729
Epoch 5/10
60000/60000 [==============================] - 3s 53us/step - loss: 0.1441 - acc: 0.9599 - val_loss: 0.0951 - val_acc: 0.9749
Epoch 6/10
60000/60000 [==============================] - 3s 53us/step - loss: 0.1357 - acc: 0.9619 - val_loss: 0.0969 - val_acc: 0.9746
Epoch 7/10
60000/60000 [==============================] - 3s 53us/step - loss: 0.1303 - acc: 0.9646 - val_loss: 0.1060 - val_acc

## Hidden Layers: 256, dropout: 0.2, batch_size : 256

In [184]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(256, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_229 (Dense)            (None, 256)               200960    
_________________________________________________________________
dropout_156 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_230 (Dense)            (None, 256)               65792     
_________________________________________________________________
dropout_157 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_231 (Dense)            (None, 10)                2570      
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________


In [185]:
history = model.fit(x_train, y_train,
                    batch_size=256,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 5s 80us/step - loss: 0.3519 - acc: 0.8936 - val_loss: 0.1712 - val_acc: 0.9453
Epoch 2/10
60000/60000 [==============================] - 2s 32us/step - loss: 0.1458 - acc: 0.9562 - val_loss: 0.1025 - val_acc: 0.9672
Epoch 3/10
60000/60000 [==============================] - 2s 32us/step - loss: 0.1025 - acc: 0.9685 - val_loss: 0.1027 - val_acc: 0.9663
Epoch 4/10
60000/60000 [==============================] - 2s 31us/step - loss: 0.0801 - acc: 0.9753 - val_loss: 0.0757 - val_acc: 0.9770
Epoch 5/10
60000/60000 [==============================] - 2s 32us/step - loss: 0.0691 - acc: 0.9788 - val_loss: 0.0763 - val_acc: 0.9762
Epoch 6/10
60000/60000 [==============================] - 2s 32us/step - loss: 0.0564 - acc: 0.9825 - val_loss: 0.0720 - val_acc: 0.9781
Epoch 7/10
60000/60000 [==============================] - 2s 32us/step - loss: 0.0494 - acc: 0.9844 - val_loss: 0.0714 - val_acc

## Optimizer: Adam, Hidden Layers: 256, dropout: 0.2

In [186]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(256, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_232 (Dense)            (None, 256)               200960    
_________________________________________________________________
dropout_158 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_233 (Dense)            (None, 256)               65792     
_________________________________________________________________
dropout_159 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_234 (Dense)            (None, 10)                2570      
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________


In [187]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 6s 95us/step - loss: 0.3102 - acc: 0.9079 - val_loss: 0.1257 - val_acc: 0.9626
Epoch 2/10
60000/60000 [==============================] - 3s 53us/step - loss: 0.1248 - acc: 0.9618 - val_loss: 0.0885 - val_acc: 0.9720
Epoch 3/10
60000/60000 [==============================] - 3s 55us/step - loss: 0.0927 - acc: 0.9715 - val_loss: 0.0866 - val_acc: 0.9724
Epoch 4/10
60000/60000 [==============================] - 3s 56us/step - loss: 0.0728 - acc: 0.9772 - val_loss: 0.0668 - val_acc: 0.9795
Epoch 5/10
60000/60000 [==============================] - 3s 57us/step - loss: 0.0594 - acc: 0.9813 - val_loss: 0.0676 - val_acc: 0.9784
Epoch 6/10
60000/60000 [==============================] - 3s 57us/step - loss: 0.0515 - acc: 0.9834 - val_loss: 0.0681 - val_acc: 0.9783
Epoch 7/10
60000/60000 [==============================] - 3s 57us/step - loss: 0.0465 - acc: 0.9851 - val_loss: 0.0589 - val_acc

## Optimizer: Adagrad, Hidden Layers: 256, dropout: 0.2

In [188]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(256, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=Adagrad(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_235 (Dense)            (None, 256)               200960    
_________________________________________________________________
dropout_160 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_236 (Dense)            (None, 256)               65792     
_________________________________________________________________
dropout_161 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_237 (Dense)            (None, 10)                2570      
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________


In [189]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 6s 100us/step - loss: 0.2625 - acc: 0.9209 - val_loss: 0.1216 - val_acc: 0.9618
Epoch 2/10
60000/60000 [==============================] - 3s 50us/step - loss: 0.1296 - acc: 0.9611 - val_loss: 0.0957 - val_acc: 0.9698
Epoch 3/10
60000/60000 [==============================] - 3s 51us/step - loss: 0.1008 - acc: 0.9690 - val_loss: 0.0839 - val_acc: 0.9737
Epoch 4/10
60000/60000 [==============================] - 3s 51us/step - loss: 0.0842 - acc: 0.9750 - val_loss: 0.0770 - val_acc: 0.9760
Epoch 5/10
60000/60000 [==============================] - 3s 51us/step - loss: 0.0742 - acc: 0.9770 - val_loss: 0.0736 - val_acc: 0.9771
Epoch 6/10
60000/60000 [==============================] - 3s 51us/step - loss: 0.0658 - acc: 0.9797 - val_loss: 0.0667 - val_acc: 0.9790
Epoch 7/10
60000/60000 [==============================] - 3s 51us/step - loss: 0.0582 - acc: 0.9822 - val_loss: 0.0646 - val_ac

## Optimizer: Adadelta, Hidden Layers: 256, dropout: 0.2

In [190]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(256, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=Adadelta(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_238 (Dense)            (None, 256)               200960    
_________________________________________________________________
dropout_162 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_239 (Dense)            (None, 256)               65792     
_________________________________________________________________
dropout_163 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_240 (Dense)            (None, 10)                2570      
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________


In [191]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 6s 107us/step - loss: 0.3287 - acc: 0.9011 - val_loss: 0.1272 - val_acc: 0.9609
Epoch 2/10
60000/60000 [==============================] - 3s 57us/step - loss: 0.1420 - acc: 0.9574 - val_loss: 0.0949 - val_acc: 0.9697
Epoch 3/10
60000/60000 [==============================] - 3s 57us/step - loss: 0.1037 - acc: 0.9687 - val_loss: 0.0789 - val_acc: 0.9746
Epoch 4/10
60000/60000 [==============================] - 3s 57us/step - loss: 0.0832 - acc: 0.9741 - val_loss: 0.0696 - val_acc: 0.9777
Epoch 5/10
60000/60000 [==============================] - 3s 57us/step - loss: 0.0686 - acc: 0.9788 - val_loss: 0.0667 - val_acc: 0.9801
Epoch 6/10
60000/60000 [==============================] - 3s 57us/step - loss: 0.0606 - acc: 0.9808 - val_loss: 0.0758 - val_acc: 0.9767
Epoch 7/10
60000/60000 [==============================] - 3s 57us/step - loss: 0.0512 - acc: 0.9839 - val_loss: 0.0661 - val_ac

## Optimizer: Nadam, Hidden Layers: 256, dropout: 0.2

In [192]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(256, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=Nadam(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_241 (Dense)            (None, 256)               200960    
_________________________________________________________________
dropout_164 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_242 (Dense)            (None, 256)               65792     
_________________________________________________________________
dropout_165 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_243 (Dense)            (None, 10)                2570      
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________


In [193]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s 116us/step - loss: 0.2612 - acc: 0.9199 - val_loss: 0.1075 - val_acc: 0.9652
Epoch 2/10
60000/60000 [==============================] - 4s 65us/step - loss: 0.1133 - acc: 0.9660 - val_loss: 0.0842 - val_acc: 0.9736
Epoch 3/10
60000/60000 [==============================] - 4s 65us/step - loss: 0.0882 - acc: 0.9725 - val_loss: 0.0720 - val_acc: 0.9782
Epoch 4/10
60000/60000 [==============================] - 4s 66us/step - loss: 0.0683 - acc: 0.9784 - val_loss: 0.0658 - val_acc: 0.9787
Epoch 5/10
60000/60000 [==============================] - 4s 61us/step - loss: 0.0609 - acc: 0.9800 - val_loss: 0.0664 - val_acc: 0.9794
Epoch 6/10
60000/60000 [==============================] - 4s 61us/step - loss: 0.0524 - acc: 0.9833 - val_loss: 0.0647 - val_acc: 0.9799
Epoch 7/10
60000/60000 [==============================] - 4s 62us/step - loss: 0.0496 - acc: 0.9840 - val_loss: 0.0738 - val_ac

# Results:

After running this battery again, it is clear that the variance between iterations is large enough that what I thought were iterative improvements in the right direction were probably due to randomness.